# Módulo 7: GANs e VAEs - Geração Sintética de Imagens## Objetivos de Aprendizagem- Compreender os fundamentos de Generative Adversarial Networks (GANs)- Entender Variational Autoencoders (VAEs) para geração de imagens- Implementar modelos de geração sintética- Analisar aplicações práticas de geração de imagens- Comparar diferentes abordagens de geração---## 7.1 Introdução à Geração Sintética de Imagens**Geração Sintética de Imagens** é o processo de criar imagens artificialmente usando modelos de machine learning, permitindo criar conteúdo visual novo e realista.![Introdução Geração Sintética](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo7/introducao_geracao_sintetica.png)### Conceitos Fundamentais**Definição:**- **Geração**: Criação de novas imagens- **Sintética**: Produzida artificialmente- **Realista**: Visualmente convincente- **Controlável**: Parâmetros ajustáveis**Aplicações:**- **Arte digital**: Criação artística- **Data augmentation**: Aumento de datasets- **Design**: Prototipagem visual- **Entretenimento**: Conteúdo para jogos/filmes### Evolução da Geração Sintética**Progressão Histórica:**- **2014**: GANs introduzidos por Ian Goodfellow- **2015**: VAEs para geração de imagens- **2016**: DCGAN para imagens de alta qualidade- **2017**: Progressive GAN para resolução crescente- **2018**: StyleGAN para controle de estilo- **2020**: Diffusion Models revolucionam o campo- **2022**: DALL-E 2 e Midjourney popularizam![Evolução Geração Sintética](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo7/evolucao_geracao_sintetica.png)**Marcos Importantes:**- **2014**: Generative Adversarial Networks - Goodfellow et al.- **2015**: Auto-Encoding Variational Bayes - Kingma & Welling- **2016**: Deep Convolutional GANs - Radford et al.- **2017**: Progressive Growing of GANs - Karras et al.- **2019**: StyleGAN - Karras et al.- **2021**: DALL-E - Ramesh et al.**Referências:**- [Generative Adversarial Networks - Goodfellow et al.](https://arxiv.org/abs/1406.2661)- [Auto-Encoding Variational Bayes - Kingma & Welling](https://arxiv.org/abs/1312.6114)

## 7.2 Demonstração Prática: GANs Simples

Vamos implementar e visualizar um GAN simples para geração de imagens:


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

class SimpleGAN:
    """Implementação de um GAN simples para demonstração"""
    
    def __init__(self, latent_dim=100, img_size=64):
        self.latent_dim = latent_dim
        self.img_size = img_size
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Inicializar modelos
        self.generator = self._build_generator()
        self.discriminator = self._build_discriminator()
        
        # Otimizadores
        self.g_optimizer = optim.Adam(self.generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
        self.d_optimizer = optim.Adam(self.discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
        
        # Loss function
        self.criterion = nn.BCELoss()
        
        # Histórico de treinamento
        self.g_losses = []
        self.d_losses = []
        
    def _build_generator(self):
        """Constrói o gerador"""
        
        class Generator(nn.Module):
            def __init__(self, latent_dim, img_size):
                super(Generator, self).__init__()
                
                # Camadas fully connected
                self.fc1 = nn.Linear(latent_dim, 256)
                self.fc2 = nn.Linear(256, 512)
                self.fc3 = nn.Linear(512, 1024)
                self.fc4 = nn.Linear(1024, img_size * img_size * 3)
                
                # Dropout
                self.dropout = nn.Dropout(0.3)
                
            def forward(self, x):
                x = F.relu(self.fc1(x))
                x = self.dropout(x)
                x = F.relu(self.fc2(x))
                x = self.dropout(x)
                x = F.relu(self.fc3(x))
                x = self.dropout(x)
                x = torch.tanh(self.fc4(x))
                
                # Reshape para imagem
                x = x.view(x.size(0), 3, self.img_size, self.img_size)
                return x
        
        return Generator(self.latent_dim, self.img_size).to(self.device)
    
    def _build_discriminator(self):
        """Constrói o discriminador"""
        
        class Discriminator(nn.Module):
            def __init__(self, img_size):
                super(Discriminator, self).__init__()
                
                # Camadas fully connected
                self.fc1 = nn.Linear(img_size * img_size * 3, 1024)
                self.fc2 = nn.Linear(1024, 512)
                self.fc3 = nn.Linear(512, 256)
                self.fc4 = nn.Linear(256, 1)
                
                # Dropout
                self.dropout = nn.Dropout(0.3)
                
            def forward(self, x):
                x = x.view(x.size(0), -1)
                x = F.leaky_relu(self.fc1(x), 0.2)
                x = self.dropout(x)
                x = F.leaky_relu(self.fc2(x), 0.2)
                x = self.dropout(x)
                x = F.leaky_relu(self.fc3(x), 0.2)
                x = self.dropout(x)
                x = torch.sigmoid(self.fc4(x))
                return x
        
        return Discriminator(self.img_size).to(self.device)
    
    def create_synthetic_data(self, num_samples=1000):
        """Cria dados sintéticos para demonstração"""
        
        # Criar imagens sintéticas simples
        images = []
        
        for i in range(num_samples):
            # Criar imagem base
            img = np.zeros((self.img_size, self.img_size, 3), dtype=np.float32)
            
            # Adicionar formas geométricas aleatórias
            if np.random.random() < 0.5:
                # Círculo
                center = (np.random.randint(20, self.img_size-20), np.random.randint(20, self.img_size-20))
                radius = np.random.randint(10, 25)
                color = (np.random.random(), np.random.random(), np.random.random())
                
                # Desenhar círculo
                y, x = np.ogrid[:self.img_size, :self.img_size]
                mask = (x - center[0])**2 + (y - center[1])**2 <= radius**2
                img[mask] = color
            
            else:
                # Retângulo
                x1 = np.random.randint(10, self.img_size-30)
                y1 = np.random.randint(10, self.img_size-30)
                x2 = x1 + np.random.randint(20, 40)
                y2 = y1 + np.random.randint(20, 40)
                color = (np.random.random(), np.random.random(), np.random.random())
                
                img[y1:y2, x1:x2] = color
            
            # Adicionar ruído
            noise = np.random.normal(0, 0.1, img.shape)
            img = np.clip(img + noise, 0, 1)
            
            images.append(img)
        
        return np.array(images)
    
    def train_step(self, real_images, batch_size):
        """Executa um passo de treinamento"""
        
        # Converter para tensor
        real_images = torch.FloatTensor(real_images).to(self.device)
        
        # Labels
        real_labels = torch.ones(batch_size, 1).to(self.device)
        fake_labels = torch.zeros(batch_size, 1).to(self.device)
        
        # Treinar Discriminador
        self.d_optimizer.zero_grad()
        
        # Imagens reais
        real_output = self.discriminator(real_images)
        d_real_loss = self.criterion(real_output, real_labels)
        
        # Imagens falsas
        noise = torch.randn(batch_size, self.latent_dim).to(self.device)
        fake_images = self.generator(noise)
        fake_output = self.discriminator(fake_images.detach())
        d_fake_loss = self.criterion(fake_output, fake_labels)
        
        # Loss total do discriminador
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        self.d_optimizer.step()
        
        # Treinar Gerador
        self.g_optimizer.zero_grad()
        
        # Gerar imagens falsas
        noise = torch.randn(batch_size, self.latent_dim).to(self.device)
        fake_images = self.generator(noise)
        fake_output = self.discriminator(fake_images)
        
        # Loss do gerador
        g_loss = self.criterion(fake_output, real_labels)
        g_loss.backward()
        self.g_optimizer.step()
        
        # Armazenar losses
        self.g_losses.append(g_loss.item())
        self.d_losses.append(d_loss.item())
        
        return g_loss.item(), d_loss.item()
    
    def train(self, real_images, epochs=100, batch_size=32):
        """Treina o GAN"""
        
        print(f"Iniciando treinamento do GAN para {epochs} épocas...")
        
        # Criar dataloader
        dataset = TensorDataset(torch.FloatTensor(real_images))
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        for epoch in range(epochs):
            epoch_g_loss = 0
            epoch_d_loss = 0
            num_batches = 0
            
            for batch_idx, (real_batch,) in enumerate(dataloader):
                g_loss, d_loss = self.train_step(real_batch, batch_size)
                epoch_g_loss += g_loss
                epoch_d_loss += d_loss
                num_batches += 1
            
            # Média das losses
            avg_g_loss = epoch_g_loss / num_batches
            avg_d_loss = epoch_d_loss / num_batches
            
            if epoch % 20 == 0:
                print(f"Época {epoch}: G Loss = {avg_g_loss:.4f}, D Loss = {avg_d_loss:.4f}")
        
        print("Treinamento concluído!")
        
        return self.g_losses, self.d_losses
    
    def generate_images(self, num_images=16):
        """Gera imagens usando o gerador treinado"""
        
        self.generator.eval()
        
        with torch.no_grad():
            noise = torch.randn(num_images, self.latent_dim).to(self.device)
            generated_images = self.generator(noise)
            
            # Converter para numpy
            generated_images = generated_images.cpu().numpy()
            
            # Normalizar para [0, 1]
            generated_images = (generated_images + 1) / 2
            generated_images = np.clip(generated_images, 0, 1)
            
        return generated_images
    
    def visualize_training(self, real_images, generated_images):
        """Visualiza o treinamento e resultados"""
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        # Imagens reais
        for i in range(6):
            axes[0, i].imshow(real_images[i])
            axes[0, i].set_title(f'Real {i+1}')
            axes[0, i].axis('off')
        
        # Imagens geradas
        for i in range(6):
            axes[1, i].imshow(generated_images[i])
            axes[1, i].set_title(f'Gerada {i+1}')
            axes[1, i].axis('off')
        
        plt.suptitle('Comparação: Imagens Reais vs Geradas', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Gráfico de losses
        plt.figure(figsize=(12, 6))
        plt.plot(self.g_losses, label='Gerador Loss', alpha=0.7)
        plt.plot(self.d_losses, label='Discriminador Loss', alpha=0.7)
        plt.title('Evolução das Losses durante o Treinamento')
        plt.xlabel('Época')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        return generated_images

def demonstrate_simple_gan():
    """Demonstra um GAN simples"""
    
    print("=== Demonstração de GAN Simples ===")
    print("\nNota: Esta demonstração usa dados sintéticos para fins educacionais.")
    print("Em aplicações reais, use datasets reais de imagens.")
    
    # Criar instância do GAN
    gan = SimpleGAN(latent_dim=100, img_size=64)
    
    # Criar dados sintéticos
    print("\nCriando dados sintéticos...")
    real_images = gan.create_synthetic_data(num_samples=1000)
    
    # Treinar GAN
    print("\nTreinando GAN...")
    g_losses, d_losses = gan.train(real_images, epochs=100, batch_size=32)
    
    # Gerar imagens
    print("\nGerando imagens...")
    generated_images = gan.generate_images(num_images=16)
    
    # Visualizar resultados
    print("\nVisualizando resultados...")
    gan.visualize_training(real_images, generated_images)
    
    return gan, real_images, generated_images

# Executar demonstração
gan_model, real_data, generated_data = demonstrate_simple_gan()

### Análise dos Resultados

**GAN Observado:**

1. **Treinamento**: Competição entre gerador e discriminador
2. **Losses**: Evolução das perdas durante treinamento
3. **Geração**: Imagens sintéticas criadas pelo gerador
4. **Qualidade**: Comparação com imagens reais

**Insights Importantes:**
- **Competição**: Gerador vs Discriminador
- **Equilíbrio**: Balanceamento das perdas
- **Qualidade**: Melhoria gradual da geração
- **Diversidade**: Variação nas imagens geradas

**Referências:**
- [Generative Adversarial Networks - Goodfellow et al.](https://arxiv.org/abs/1406.2661)


## 7.3 Generative Adversarial Networks (GANs)**GANs** são arquiteturas que consistem em dois modelos neurais competindo: um gerador que cria imagens e um discriminador que tenta distinguir entre imagens reais e sintéticas.![GANs Conceito](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo7/gans_conceito.png)### Arquitetura GAN**Componentes Principais:****1. Gerador (Generator):**- **Entrada**: Vetor de ruído aleatório (latent space)- **Saída**: Imagem sintética- **Objetivo**: Criar imagens que enganem o discriminador- **Arquitetura**: Rede neural com camadas convolucionais**2. Discriminador (Discriminator):**- **Entrada**: Imagem (real ou sintética)- **Saída**: Probabilidade de ser real- **Objetivo**: Distinguir entre imagens reais e sintéticas- **Arquitetura**: Rede neural classificadora![Arquitetura GAN](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo7/arquitetura_gan.png)### Processo de Treinamento**1. Treinamento do Discriminador:**- **Imagens reais**: Label = 1 (real)- **Imagens sintéticas**: Label = 0 (falso)- **Loss**: Binary Cross Entropy- **Objetivo**: Maximizar precisão de classificação**2. Treinamento do Gerador:**- **Entrada**: Ruído aleatório- **Saída**: Imagem sintética- **Loss**: Discriminador classifica como real- **Objetivo**: Minimizar perda do discriminador**3. Competição Adversarial:**- **Equilíbrio**: Gerador e discriminador competem- **Convergência**: Ambos melhoram simultaneamente- **Estabilidade**: Desafio principal dos GANs- **Resultado**: Imagens de alta qualidade### Tipos de GANs**1. DCGAN (Deep Convolutional GAN):**- **Arquitetura**: Convolucional para gerador e discriminador- **Inovações**: Batch normalization, ReLU, LeakyReLU- **Aplicação**: Imagens de alta qualidade- **Referência**: Radford et al. (2016)**2. Progressive GAN:**- **Conceito**: Treinamento progressivo de baixa para alta resolução- **Vantagem**: Estabilidade e qualidade- **Aplicação**: Imagens de alta resolução- **Referência**: Karras et al. (2017)**3. StyleGAN:**- **Inovação**: Controle de estilo e conteúdo- **Arquitetura**: Mapeamento de ruído para estilo- **Aplicação**: Geração controlada de faces- **Referência**: Karras et al. (2019)![Tipos de GANs](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo7/tipos_gans.png)**Referências:**- [Generative Adversarial Networks - Goodfellow et al.](https://arxiv.org/abs/1406.2661)- [Unsupervised Representation Learning with Deep Convolutional GANs - Radford et al.](https://arxiv.org/abs/1511.06434)

## 7.4 Variational Autoencoders (VAEs)**VAEs** são modelos generativos que aprendem a representar dados em um espaço latente de baixa dimensionalidade, permitindo geração de novas amostras.![VAEs Conceito](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo7/vaes_conceito.png)### Arquitetura VAE**Componentes Principais:****1. Encoder:**- **Entrada**: Imagem real- **Saída**: Parâmetros da distribuição latente (μ, σ)- **Objetivo**: Comprimir imagem para espaço latente- **Arquitetura**: Rede neural com camadas convolucionais**2. Decoder:**- **Entrada**: Amostra do espaço latente- **Saída**: Imagem reconstruída- **Objetivo**: Reconstruir imagem a partir do espaço latente- **Arquitetura**: Rede neural com camadas transpostas![Arquitetura VAE](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo7/arquitetura_vae.png)### Processo de Treinamento**1. Forward Pass:**- **Encoder**: Imagem → μ, σ- **Sampling**: z ~ N(μ, σ²)- **Decoder**: z → Imagem reconstruída**2. Loss Function:**- **Reconstruction Loss**: Diferença entre imagem original e reconstruída- **KL Divergence**: Regularização do espaço latente- **Total Loss**: Reconstruction + β × KL Divergence**3. Regularização:**- **KL Divergence**: Força distribuição latente próxima a N(0,1)- **β-VAE**: Controle do trade-off entre reconstrução e regularização- **Resultado**: Espaço latente contínuo e interpretável### Demonstração Prática: VAE SimplesVamos implementar e visualizar um VAE simples:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

class SimpleVAE:
    """Implementação de um VAE simples para demonstração"""
    
    def __init__(self, latent_dim=2, img_size=64):
        self.latent_dim = latent_dim
        self.img_size = img_size
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Inicializar modelo
        self.model = self._build_vae()
        
        # Otimizador
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        
        # Histórico de treinamento
        self.losses = []
        self.recon_losses = []
        self.kl_losses = []
        
    def _build_vae(self):
        """Constrói o VAE"""
        
        class VAE(nn.Module):
            def __init__(self, latent_dim, img_size):
                super(VAE, self).__init__()
                
                self.latent_dim = latent_dim
                self.img_size = img_size
                
                # Encoder
                self.encoder = nn.Sequential(
                    nn.Linear(img_size * img_size * 3, 512),
                    nn.ReLU(),
                    nn.Linear(512, 256),
                    nn.ReLU(),
                    nn.Linear(256, 128),
                    nn.ReLU()
                )
                
                # Mean e log variance
                self.fc_mu = nn.Linear(128, latent_dim)
                self.fc_logvar = nn.Linear(128, latent_dim)
                
                # Decoder
                self.decoder = nn.Sequential(
                    nn.Linear(latent_dim, 128),
                    nn.ReLU(),
                    nn.Linear(128, 256),
                    nn.ReLU(),
                    nn.Linear(256, 512),
                    nn.ReLU(),
                    nn.Linear(512, img_size * img_size * 3),
                    nn.Sigmoid()
                )
                
            def encode(self, x):
                """Codifica imagem para espaço latente"""
                x = x.view(x.size(0), -1)
                h = self.encoder(x)
                mu = self.fc_mu(h)
                logvar = self.fc_logvar(h)
                return mu, logvar
                
            def reparameterize(self, mu, logvar):
                """Reparameterization trick"""
                std = torch.exp(0.5 * logvar)
                eps = torch.randn_like(std)
                return mu + eps * std
                
            def decode(self, z):
                """Decodifica espaço latente para imagem"""
                x = self.decoder(z)
                return x.view(x.size(0), 3, self.img_size, self.img_size)
                
            def forward(self, x):
                mu, logvar = self.encode(x)
                z = self.reparameterize(mu, logvar)
                recon_x = self.decode(z)
                return recon_x, mu, logvar
        
        return VAE(self.latent_dim, self.img_size).to(self.device)
    
    def vae_loss(self, recon_x, x, mu, logvar):
        """Calcula a loss do VAE"""
        
        # Reconstruction loss (MSE)
        recon_loss = F.mse_loss(recon_x, x, reduction='sum')
        
        # KL divergence
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        
        # Total loss
        total_loss = recon_loss + kl_loss
        
        return total_loss, recon_loss, kl_loss
    
    def train_step(self, batch):
        """Executa um passo de treinamento"""
        
        self.optimizer.zero_grad()
        
        # Forward pass
        recon_batch, mu, logvar = self.model(batch)
        
        # Calcular loss
        loss, recon_loss, kl_loss = self.vae_loss(recon_batch, batch, mu, logvar)
        
        # Backward pass
        loss.backward()
        self.optimizer.step()
        
        return loss.item(), recon_loss.item(), kl_loss.item()
    
    def train(self, data, epochs=100, batch_size=32):
        """Treina o VAE"""
        
        print(f"Iniciando treinamento do VAE para {epochs} épocas...")
        
        # Criar dataloader
        dataset = TensorDataset(torch.FloatTensor(data))
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        for epoch in range(epochs):
            epoch_loss = 0
            epoch_recon_loss = 0
            epoch_kl_loss = 0
            num_batches = 0
            
            for batch_idx, (batch,) in enumerate(dataloader):
                batch = batch.to(self.device)
                loss, recon_loss, kl_loss = self.train_step(batch)
                
                epoch_loss += loss
                epoch_recon_loss += recon_loss
                epoch_kl_loss += kl_loss
                num_batches += 1
            
            # Média das losses
            avg_loss = epoch_loss / num_batches
            avg_recon_loss = epoch_recon_loss / num_batches
            avg_kl_loss = epoch_kl_loss / num_batches
            
            # Armazenar losses
            self.losses.append(avg_loss)
            self.recon_losses.append(avg_recon_loss)
            self.kl_losses.append(avg_kl_loss)
            
            if epoch % 20 == 0:
                print(f"Época {epoch}: Loss = {avg_loss:.4f}, Recon = {avg_recon_loss:.4f}, KL = {avg_kl_loss:.4f}")
        
        print("Treinamento concluído!")
        
        return self.losses, self.recon_losses, self.kl_losses
    
    def generate_images(self, num_images=16):
        """Gera imagens usando o VAE treinado"""
        
        self.model.eval()
        
        with torch.no_grad():
            # Amostrar do espaço latente
            z = torch.randn(num_images, self.latent_dim).to(self.device)
            generated_images = self.model.decode(z)
            
            # Converter para numpy
            generated_images = generated_images.cpu().numpy()
            
        return generated_images
    
    def reconstruct_images(self, images):
        """Reconstrói imagens usando o VAE"""
        
        self.model.eval()
        
        with torch.no_grad():
            images_tensor = torch.FloatTensor(images).to(self.device)
            recon_images, _, _ = self.model(images_tensor)
            
            # Converter para numpy
            recon_images = recon_images.cpu().numpy()
            
        return recon_images
    
    def visualize_results(self, original_images, reconstructed_images, generated_images):
        """Visualiza resultados do VAE"""
        
        fig, axes = plt.subplots(3, 6, figsize=(18, 9))
        
        # Imagens originais
        for i in range(6):
            axes[0, i].imshow(original_images[i])
            axes[0, i].set_title(f'Original {i+1}')
            axes[0, i].axis('off')
        
        # Imagens reconstruídas
        for i in range(6):
            axes[1, i].imshow(reconstructed_images[i])
            axes[1, i].set_title(f'Reconstruída {i+1}')
            axes[1, i].axis('off')
        
        # Imagens geradas
        for i in range(6):
            axes[2, i].imshow(generated_images[i])
            axes[2, i].set_title(f'Gerada {i+1}')
            axes[2, i].axis('off')
        
        plt.suptitle('VAE: Originais, Reconstruídas e Geradas', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Gráfico de losses
        plt.figure(figsize=(12, 6))
        plt.plot(self.losses, label='Total Loss', alpha=0.7)
        plt.plot(self.recon_losses, label='Reconstruction Loss', alpha=0.7)
        plt.plot(self.kl_losses, label='KL Divergence Loss', alpha=0.7)
        plt.title('Evolução das Losses durante o Treinamento')
        plt.xlabel('Época')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        return generated_images

def demonstrate_simple_vae():
    """Demonstra um VAE simples"""
    
    print("=== Demonstração de VAE Simples ===")
    print("\nNota: Esta demonstração usa dados sintéticos para fins educacionais.")
    print("Em aplicações reais, use datasets reais de imagens.")
    
    # Criar instância do VAE
    vae = SimpleVAE(latent_dim=2, img_size=64)
    
    # Criar dados sintéticos
    print("\nCriando dados sintéticos...")
    real_images = vae.create_synthetic_data(num_samples=1000)
    
    # Treinar VAE
    print("\nTreinando VAE...")
    losses, recon_losses, kl_losses = vae.train(real_images, epochs=100, batch_size=32)
    
    # Reconstruir imagens
    print("\nReconstruindo imagens...")
    reconstructed_images = vae.reconstruct_images(real_images[:6])
    
    # Gerar imagens
    print("\nGerando imagens...")
    generated_images = vae.generate_images(num_images=16)
    
    # Visualizar resultados
    print("\nVisualizando resultados...")
    vae.visualize_results(real_images[:6], reconstructed_images, generated_images)
    
    return vae, real_images, reconstructed_images, generated_images

# Executar demonstração
vae_model, real_data, recon_data, generated_data = demonstrate_simple_vae()

### Análise dos Resultados

**VAE Observado:**

1. **Reconstrução**: Imagens reconstruídas a partir do espaço latente
2. **Geração**: Novas imagens geradas do espaço latente
3. **Losses**: Evolução das perdas durante treinamento
4. **Qualidade**: Comparação entre originais e reconstruídas

**Insights Importantes:**
- **Reconstrução**: VAE pode reconstruir imagens
- **Geração**: Espaço latente permite geração
- **Regularização**: KL divergence controla o espaço latente
- **Trade-off**: Reconstrução vs regularização

**Referências:**
- [Auto-Encoding Variational Bayes - Kingma & Welling](https://arxiv.org/abs/1312.6114)


## 7.5 Comparação: GANs vs VAEs### Diferenças Fundamentais**GANs:**- **Objetivo**: Gerar imagens realistas- **Arquitetura**: Gerador + Discriminador- **Treinamento**: Competição adversarial- **Qualidade**: Imagens de alta qualidade- **Controle**: Limitado**VAEs:**- **Objetivo**: Aprender representação latente- **Arquitetura**: Encoder + Decoder- **Treinamento**: Minimização de loss- **Qualidade**: Imagens mais suaves- **Controle**: Espaço latente interpretável![Comparação GANs vs VAEs](https://raw.githubusercontent.com/rfapo/visao-computacional/main/images/modulo7/comparacao_gans_vaes.png)### Vantagens e Desvantagens**GANs - Vantagens:**- **Qualidade**: Imagens muito realistas- **Diversidade**: Grande variedade de saídas- **Inovação**: Arquiteturas criativas- **Aplicação**: Arte, entretenimento**GANs - Desvantagens:**- **Estabilidade**: Difícil de treinar- **Controle**: Limitado controle sobre saída- **Modo Collapse**: Problema comum- **Recursos**: Requer muitos recursos**VAEs - Vantagens:**- **Estabilidade**: Treinamento estável- **Controle**: Espaço latente interpretável- **Reconstrução**: Pode reconstruir imagens- **Regularização**: Espaço latente regularizado**VAEs - Desvantagens:**- **Qualidade**: Imagens mais suaves- **Blur**: Efeito de desfoque- **Limitação**: Espaço latente limitado- **Trade-off**: Reconstrução vs geração### Aplicações Práticas**GANs - Casos de Uso:**- **Arte digital**: Criação artística- **Data augmentation**: Aumento de datasets- **Super-resolução**: Aumento de resolução- **Transferência de estilo**: Aplicação de estilos**VAEs - Casos de Uso:**- **Compressão**: Compressão de imagens- **Anomaly detection**: Detecção de anomalias- **Representação**: Aprendizado de representações- **Interpolação**: Interpolação no espaço latente**Referências:**- [Generative Adversarial Networks - Goodfellow et al.](https://arxiv.org/abs/1406.2661)- [Auto-Encoding Variational Bayes - Kingma & Welling](https://arxiv.org/abs/1312.6114)

## Resumo do Módulo 7

### Principais Conceitos Abordados

1. **Introdução à Geração Sintética**
   - Conceitos fundamentais
   - Evolução histórica
   - Aplicações práticas

2. **Generative Adversarial Networks (GANs)**
   - Arquitetura e componentes
   - Processo de treinamento
   - Tipos e variações

3. **Variational Autoencoders (VAEs)**
   - Arquitetura encoder-decoder
   - Espaço latente
   - Regularização KL

4. **Comparação e Aplicações**
   - Diferenças fundamentais
   - Vantagens e desvantagens
   - Casos de uso práticos

### Demonstrações Práticas

**1. GAN Simples:**
   - Implementação completa
   - Treinamento adversarial
   - Geração de imagens

**2. VAE Simples:**
   - Arquitetura encoder-decoder
   - Reconstrução e geração
   - Análise do espaço latente

### Próximos Passos

No próximo módulo, exploraremos **Vision Transformers e Mecanismos de Atenção**, onde aprenderemos sobre a revolução dos transformers na visão computacional.

### Referências Principais

- [Generative Adversarial Networks - Goodfellow et al.](https://arxiv.org/abs/1406.2661)
- [Auto-Encoding Variational Bayes - Kingma & Welling](https://arxiv.org/abs/1312.6114)
- [Unsupervised Representation Learning with Deep Convolutional GANs - Radford et al.](https://arxiv.org/abs/1511.06434)

---

**Próximo Módulo**: Vision Transformers e Mecanismos de Atenção
